In [1]:
import os
import sys
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
#absolutePath = input('Enter the path location: ')
absolutePath = r'C:\Users\gulce\Desktop\EEE 8TH SEMESTER\CS 464\Homeworks\HW2\afhq_dog'
# Check if the OS path of the folder exists
try:
    X = np.empty((len(os.listdir(absolutePath)), 4096, 3))
    print('Folder path is correct!!!')
except Exception:
    print('Invalid folder path!!!')
    sys.exit(1)

In [ ]:
imgCount = 0
for fileName in os.listdir(absolutePath):  # read each image in the folder and store them into a numpy array
    imagePath = os.path.join(absolutePath, fileName)
    # Check if the OS path of the images are valid
    try:
        image = Image.open(imagePath)
    except Exception:
        print('Invalid image path!!!')
        sys.exit(1)
    image = np.array(image.resize((64, 64), Image.Resampling.BILINEAR), dtype=int)
    X[imgCount] = image.reshape((4096, 3))
    imgCount += 1
# Split image array into its R, G, and B components
X0 = X[:, :, 0]  # red
X1 = X[:, :, 1]  # green
X2 = X[:, :, 2]  # blue

In [ ]:
import numpy as np


class PCA:
    def __init__(self):
        self.numPC = None
        self.eigenValues = None
        self.eigenVectors = None

    def getEigen(self, X):
        # Get the sorted (descending order) eigenvalues and eigenvectors from a feature array
        normalized = X - X.mean(axis=0, keepdims=True)  # subtract the mean
        covariance = np.cov(normalized, rowvar=False)  # find covariance matrix
        self.eigenValues, self.eigenVectors = np.linalg.eig(covariance)  # find eigenvalues and eigenvectors of the covariance matrix
        eigenIndices = np.argsort(-self.eigenValues)  # find the order of the sorted eigenvalues with their indices
        self.eigenVectors = self.eigenVectors.T  # get the transpose of eigenvectors
        self.eigenValues = self.eigenValues[eigenIndices]  # sort the eigenvalues in descending order
        self.eigenVectors = self.eigenVectors[eigenIndices]  # sort the eigenvectors according to eigenvalues

    def getkLargestPCs(self, k):
        # Get the first k largest eigenvalues and their eigenvectors
        self.numPC = k
        largestEigenValues = self.eigenValues[:self.numPC]
        largestEigenVectors = self.eigenVectors[:self.numPC]
        return largestEigenValues, largestEigenVectors

    def getPVE(self, largestEigenValues):
        # Get the PVE of the principle components
        listPVE = []
        for e in largestEigenValues:
            listPVE.append(e / np.sum(self.eigenValues))
        return listPVE

    def getNormalizedPCs(self, largestEigenVectors):
        # Get the normalized version of the principal components
        X_PCs = np.reshape(largestEigenVectors, (self.numPC, 64, 64))
        normalizedX_PCs = (X_PCs - X_PCs.min()) / (X_PCs.max() - X_PCs.min())
        return normalizedX_PCs

In [ ]:
def plotPVE(listPVE, colorName):
    # Plot the PVE vs PC graph
    title = 'PVE vs. Principal Component for ' + colorName
    plt.title(title)
    plt.xlabel('Principal Component')
    plt.ylabel('Portion of Variance Explained')
    x = np.arange(1, len(listPVE)+1, 1)
    plt.plot(x, listPVE, marker='o')
    plt.show()

In [ ]:
def getKmin(pca):
    # Find the minimum number of PCs that are required to obtain at least 70% PVE
    k = 1
    sumPVE = 0
    while sumPVE < 0.70:
        eigenValuesL, eigenVectorsL = pca.getkLargestPCs(k)
        listPVE = pca.getPVE(eigenValuesL)
        sumPVE = np.sum(listPVE)
        k += 1
    return k

In [ ]:
pca_X0 = PCA()  # PCA for red
pca_X1 = PCA()  # PCA for green
pca_X2 = PCA()  # PCA for blue

In [ ]:
pca_X0.getEigen(X0)
eigenValuesL_X0, eigenVectorsL_X0 = pca_X0.getkLargestPCs(10)  # get first 10 eigenvalues and eigenvectors for red
listPVE_X0 = pca_X0.getPVE(eigenValuesL_X0)  # get the list of first 10 PVE for red
print('PVEs of the first 10 PCs for red: ', listPVE_X0)
print('Sum of the first 10 PCs for red: ', np.sum(listPVE_X0))
plotPVE(listPVE_X0, 'Red')

In [ ]:
pca_X1.getEigen(X1)
eigenValuesL_X1, eigenVectorsL_X1 = pca_X1.getkLargestPCs(10)  # get 10 eigenvalues and eigenvectors for green
listPVE_X1 = pca_X1.getPVE(eigenValuesL_X1)  # get the list of first 10 PVE for green
print('PVEs of the first 10 PCs for green: ', listPVE_X1)
print('Sum of the first 10 PCs for green: ', np.sum(listPVE_X1))
plotPVE(listPVE_X0, 'Green')

In [ ]:
pca_X2.getEigen(X2)
eigenValuesL_X2, eigenVectorsL_X2 = pca_X2.getkLargestPCs(10)  # get 10 eigenvalues and eigenvectors for blue
listPVE_X2 = pca_X2.getPVE(eigenValuesL_X2)  # get the list of first 10 PVE for blue
print('PVEs of the first 10 PCs for blue: ', listPVE_X2)
print('Sum of the first 10 PCs for blue: ', np.sum(listPVE_X2))
plotPVE(listPVE_X0, 'Blue')

In [ ]:
normalizedX0_PCs = pca_X0.getNormalizedPCs(eigenVectorsL_X0)
normalizedX1_PCs = pca_X1.getNormalizedPCs(eigenVectorsL_X1)
normalizedX2_PCs = pca_X2.getNormalizedPCs(eigenVectorsL_X2)
normalized_PCs = np.stack((normalizedX0_PCs, normalizedX1_PCs, normalizedX2_PCs), axis=3)

In [ ]:
k_X0 = getKmin(pca_X0)
print('The minimum number of PCs that are required to obtain at least 70% PVE for red is', k_X0)
k_X1 = getKmin(pca_X1)
print('The minimum number of PCs that are required to obtain at least 70% PVE for green is', k_X1)
k_X2 = getKmin(pca_X2)
print('The minimum number of PCs that are required to obtain at least 70% PVE for blue is', k_X2)
print('The minimum number of PCs that are required to obtain at least 70% PVE for all channels is',
      max(k_X0, k_X1, k_X2))

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(16, 8))
for i in range(0, 5):
    axs[0, i].title.set_text('PC ' + str(i+1))
    axs[0, i].imshow(normalized_PCs[i])
for i in range(0, 5):
    axs[1, i].title.set_text('PC ' + str(i+6))
    axs[1, i].imshow(normalized_PCs[i + 5])
plt.show()

In [ ]:
def perform(k, originalImg):
    # Reconstruct the original image with k principal components

    # Find k eigenvalues and eigenvectors for all channels
    eigenValuesL_X0, eigenVectorsL_X0 = pca_X0.getkLargestPCs(k)
    eigenValuesL_X1, eigenVectorsL_X1 = pca_X1.getkLargestPCs(k)
    eigenValuesL_X2, eigenVectorsL_X2 = pca_X2.getkLargestPCs(k)

    # Arrange original image for all channels
    originalImgX0 = originalImg[:, 0]
    originalImgX1 = originalImg[:, 1]
    originalImgX2 = originalImg[:, 2]

    nOriginalImgX0 = originalImgX0 - originalImgX0.mean(axis=0, keepdims=True)
    nOriginalImgX1 = originalImgX1 - originalImgX1.mean(axis=0, keepdims=True)
    nOriginalImgX2 = originalImgX2 - originalImgX2.mean(axis=0, keepdims=True)

    # Find reconstructed image for all channels
    dotProduct_X0 = (np.dot(np.dot(eigenVectorsL_X0, nOriginalImgX0), eigenVectorsL_X0)
                     + originalImgX0.mean(axis=0, keepdims=True)).reshape((64, 64))
    dotProduct_X1 = (np.dot(np.dot(eigenVectorsL_X1, nOriginalImgX1), eigenVectorsL_X1)
                     + originalImgX1.mean(axis=0, keepdims=True)).reshape((64, 64))
    dotProduct_X2 = (np.dot(np.dot(eigenVectorsL_X2, nOriginalImgX2), eigenVectorsL_X2)
                     + originalImgX2.mean(axis=0, keepdims=True)).reshape((64, 64))

    dotProduct = np.stack((dotProduct_X0, dotProduct_X1, dotProduct_X2), axis=2)
    return dotProduct

In [ ]:
# imgPath = input('Enter the path location of the image: ')
imgPath = r'C:\Users\gulce\Desktop\EEE 8TH SEMESTER\CS 464\Homeworks\HW2\afhq_dog\flickr_dog_000002.jpg'
# Check if the OS path of the image is valid
try:
    originalImage = Image.open(imgPath)
    print('Image path is correct!!!')
except Exception:
    print('Invalid image path!!!')
    sys.exit(1)
# Plot the original image
plt.figure()
plt.title('Original Image')
plt.imshow(originalImage)
plt.show()

In [ ]:
originalImg = np.array(originalImage.resize((64, 64), Image.Resampling.BILINEAR))
# Plot the reshaped original image to 64x64 pixels
plt.figure()
plt.title('Original Image in 64x64 pixels')
plt.imshow(originalImg)
plt.show()

In [ ]:
originalImg = originalImg.reshape((4096, 3))
k_list = [1, 50, 250, 500, 1000, 4096]  # list of the number of principal components

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(8, 16))
r = 0
c = 0
for k in k_list:
    reconstructedImage = perform(k, originalImg)  # find the reconstructed image for k principal component
    axs[r, c].imshow(reconstructedImage.astype('uint8'))
    axs[r, c].title.set_text('Reconstructed Image with ' + str(k) + 'PCs')
    r += 1
    if r == 3:
        r = 0
        c = 1
plt.show()